In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Setup

In [3]:
mer = pd.ExcelFile('dataset/2. MER PointTransaction 2023_RetailsResellers_Final.xlsx')
mer_sheet1 = pd.read_excel(mer, 'MER Transaction')
mer_sheet2 = pd.read_excel(mer, 'Point Per Litre')

In [4]:
print(mer_sheet1.shape)
mer_sheet1.head()

(9429, 8)


,Invoicedate,Distributor,GroupId,Type,Provincename,MatNo,Material Name,New_Volume(L)
0,2023-02-14,A,ERW_AI8N2021070006,Retails,Pathum Thani,123150,"MOBIL MULTIPURPOSE ATF, 208LT DR",7168
1,2023-04-20,D,ERW_DBG72020110082,Reseller,Songkhla,124213,MOBIL ATF 220 D 208LT,912
2,2023-10-19,D,ERW_DBG72020110082,Reseller,Songkhla,124216,MOBIL ATF 220 P1/20LT,7340
3,2023-07-25,D,ERW_DBG72020110082,Reseller,Songkhla,124216,MOBIL ATF 220 P1/20LT,1640
4,2023-10-12,D,ERW_DBG72020110082,Reseller,Songkhla,124216,MOBIL ATF 220 P1/20LT,7080


In [5]:
print(mer_sheet2.shape)
mer_sheet2.head()

(145, 2)


,MatNo,Points
0,144673,10
1,140521,10
2,144473,10
3,145832,10
4,140418,10


In [6]:
mer = pd.merge(mer_sheet1, mer_sheet2, on='MatNo', how='left')
print(mer.shape)
mer.head()

(9429, 9)


,Invoicedate,Distributor,GroupId,Type,Provincename,MatNo,Material Name,New_Volume(L),Points
0,2023-02-14,A,ERW_AI8N2021070006,Retails,Pathum Thani,123150,"MOBIL MULTIPURPOSE ATF, 208LT DR",7168,1
1,2023-04-20,D,ERW_DBG72020110082,Reseller,Songkhla,124213,MOBIL ATF 220 D 208LT,912,1
2,2023-10-19,D,ERW_DBG72020110082,Reseller,Songkhla,124216,MOBIL ATF 220 P1/20LT,7340,1
3,2023-07-25,D,ERW_DBG72020110082,Reseller,Songkhla,124216,MOBIL ATF 220 P1/20LT,1640,1
4,2023-10-12,D,ERW_DBG72020110082,Reseller,Songkhla,124216,MOBIL ATF 220 P1/20LT,7080,1


Handle null/ missing values

In [7]:
mer.isnull().sum()

Invoicedate      0
Distributor      0
GroupId          0
Type             0
Provincename     0
MatNo            0
Material Name    0
New_Volume(L)    0
Points           0
dtype: int64

In [8]:
# Fix Typos
mat_default = mer_sheet1.sort_values(['MatNo', 'Material Name']).groupby('MatNo').head(1)\
                        .loc[:, ['MatNo', 'Material Name']].set_index('MatNo')
mat_default['Material Name'] = mat_default['Material Name'].str.replace('โมบิล', 'MOBIL')
mat_default['Material Name'] = mat_default['Material Name'].str.replace('เดลแวค', 'DELVAC')
mat_default['Material Name'] = mat_default['Material Name'].str.replace('สเปเชียล', 'SPECIAL')
mat_default.loc[ mat_default.index == 140563, 'Material Name'] = 'MOBIL DELVAC 1 GEAR OIL 75W90, 1/35LB'

mat_default.shape
# with pd.option_context('display.max_rows', None):
#     display(mat_default)

(145, 1)

In [9]:
for i in mer['MatNo'].unique():
    mer.loc[mer['MatNo'] == i, 'Material Name'] = mat_default.loc[i, 'Material Name']
            
mer[ ['MatNo', 'Material Name']].drop_duplicates().shape

(145, 2)

In [10]:
mer.dropna(inplace=True)
mer.isnull().any()

Invoicedate      False
Distributor      False
GroupId          False
Type             False
Provincename     False
MatNo            False
Material Name    False
New_Volume(L)    False
Points           False
dtype: bool

### EDA
Points per group

In [11]:
mer['Provincename'] = mer['Provincename'].replace('N\A', np.nan)
mer[mer['Provincename'].isnull()].head()

,Invoicedate,Distributor,GroupId,Type,Provincename,MatNo,Material Name,New_Volume(L),Points
1705,2023-11-03,A,ERW_A7QF2022060002,Retails,NaN,140043,"MOBIL ATF 3309 , 12x1 LT CTN",5392,1
1845,2023-02-13,A,ERW_A7QF2022060003,Retails,NaN,140418,MOBIL 1 TBD 5W-40 4X6L/CAR,5088,10
1910,2023-02-13,A,ERW_A7QF2022060003,Retails,NaN,140521,"MOBIL 1 0W-40 (SN), 4x4LT CTN",608,10
2260,2023-02-13,A,ERW_A7QF2022060003,Retails,NaN,142923,MOBIL SPECIAL 20W-50 4X4L/CAR,6544,1
2275,2023-10-26,A,ERW_A7QF2022060003,Retails,NaN,142923,MOBIL SPECIAL 20W-50 4X4L/CAR,1104,1


**Churn Detection**\
hypothesis: if trending of volume tend to decrease over year, we will classify as 'Churn'

In [12]:
mer['quarter'] = mer['Invoicedate'].dt.quarter
volume_by_quarter = mer.groupby(['GroupId', 'MatNo', 'quarter'])\
                        .agg({'New_Volume(L)': 'sum'})
volume_by_quarter = volume_by_quarter.sort_values(by=['GroupId', 'MatNo', 'quarter']).reset_index()
volume_by_quarter.head(5)

,GroupId,MatNo,quarter,New_Volume(L)
0,ERW_A1DS2023040002,143874,3,2328
1,ERW_A1DS2023040002,143874,4,5532
2,ERW_A1DS2023040002,143877,2,12544
3,ERW_A1DS2023040002,143877,3,8120
4,ERW_A1DS2023040002,143877,4,5040


In [13]:
churn_df = pd.pivot_table(data=volume_by_quarter, values='New_Volume(L)', 
               index=['GroupId', 'MatNo'], columns='quarter').reset_index()
churn_df = churn_df.fillna(0)
churn_df['volume_diff'] = 0

# iterate over quarter for calculating volume trending
for ind in churn_df.index:
    result = 0
    for quarter in [4, 3, 2]:
        for i in reversed(range(1, quarter)):
            temp1 = churn_df.loc[ind, quarter]  # quarter หลัง 
            temp2 = churn_df.loc[ind, i]  # quarter ก่อน
            if temp1 != 0 and temp2 != 0:
                result += temp1 - temp2
                break
            else:
                continue
        
        churn_df.loc[ind, 'volume_diff'] = result
        
# Churn classification
churn_df['Churn'] = ' '        
# Check quarter order
for ind in churn_df[churn_df['volume_diff'] == 0].index:
    q_check = []
    for quarter in [1, 2, 3, 4]:
        if churn_df.loc[ind, quarter] != 0:
            q_check.append(quarter)
        else:
            continue
        
    if q_check.count(4) != 0:  # have order in quarter 4
        churn_df.loc[ind, 'Churn'] = 'No'
    else:  
        # ซื้อ 2 quarter ขึ้นไป
        if len(q_check) >= 2:
            churn_df.loc[ind, 'Churn'] = 'No'
        else:
            churn_df.loc[ind, 'Churn'] = 'Yes'

# check by sales consecutive different
for ind in churn_df[ churn_df['volume_diff'] != 0 ].index:
    if churn_df.loc[ind, 'volume_diff'] < 0:
        churn_df.loc[ind, 'Churn'] = ' Yes'
    else:
        churn_df.loc[ind, 'Churn'] = 'No'

churn_df.head()

quarter,GroupId,MatNo,1,2,3,4,volume_diff,Churn
0,ERW_A1DS2023040002,143874,0.0,0.0,2328.0,5532.0,3204,No
1,ERW_A1DS2023040002,143877,0.0,12544.0,8120.0,5040.0,-7504,Yes
2,ERW_A1DS2023040002,145826,0.0,7160.0,0.0,4580.0,-2580,Yes
3,ERW_A1DS2023040003,145472,6076.0,0.0,0.0,0.0,0,Yes
4,ERW_A1DS2023040003,145474,0.0,1344.0,0.0,0.0,0,Yes


In [ ]:
mer.to_csv('clean_data/MER_clean.csv', index=False)
churn_df.to_csv('Machine_learning/churn.csv', index=False)